## Задача на программирование: точки и отрезки

В первой строке задано два целых числа $1 \le n \le 50000$ и $1 \le n \le 50000$ — количество отрезков и точек на прямой, соответственно. Следующие $n$ строк содержат по два целых числа $a_i$ и $b_i$ ($a_i \le b_i$) — координаты концов отрезков. Последняя строка содержит m целых чисел — координаты точек. Все координаты не превышают $10^8$ по модулю. Точка считается принадлежащей отрезку, если она находится внутри него или на границе. Для каждой точки в порядке появления во вводе выведите, скольким отрезкам она принадлежит.

### решение

In [1]:
%%writefile dots_in_cuts.py
from sys import stdin
read = lambda: map(int, stdin.readline().split())

n, m = read()
axis = []

for i in range(n):
    l, r = read()
    axis.append((l, -1, i))
    axis.append((r, 1, -i))

for i, d in enumerate(read()):
    axis.append((d, 0, i))

axis.sort()
hits = [0] * m
depth = 0

for x, dec, i in axis:
    depth -= dec
    if not dec:
        hits[i] = depth

print(*hits)

Overwriting dots_in_cuts.py


In [2]:
%%bash
python3 dots_in_cuts.py << EOF
2 3
0 5
7 10
1 6 11
EOF

1 0 0


Всё это дело можно свести к единственной сортировке массива размером $m+2n$.

Берём числовую прямую `axis` и размещаем на ней три вида объектов: начала отрезков, концы отрезков и точки. После чтения `stdin` вся эта куча точек неупорядочена. Отсортируем её по координате $x$. Теперь сделаем по ней один единственный проход слева направо. Сначала предположим для простоты, что никакие две координаты точек, начал и концов отрезков не совпадают. Когда мы встречаем очередной объект, то смотрим: если это начало отрезка, значит мы немного заглубились в отрезки -- увеличиваем вложенность на 1. Если это конец отрезка, уменьшаем текущую вложенность отрезков на 1. Если это точка, то выведем текущее значение вложенности для этой точки.

Если на входе вдруг попадутся невалидные отрезки, у которых левый конец больше правого, то они могут усложнить порядок расчёта глубины. Просто отбросим их при вводе.

Теперь немного доработаем алгоритм, чтобы он не путался, когда координаты точек и концов отрезков совпадают. Нам надо, чтобы при их совпадении всегда был такой порядок: левый конец отрезка, точка, правый конец отрезка (а иначе отрезок для точки, лежащей точно на его конце, не посчитается). Для этого будем класть в массив не просто координату, а кортеж: (координата,тип_объекта), причём тип левого конца (-1) будет при сравнении меньше типа точки (0) и меньше правого конца (1).

Осталось учесть случай, когда отрезки вкладываются друг в друга, и координаты их концов совпадают: нам надо, чтобы если отрезок А открылся раньше, то чтобы он позже закрылся. Для этого добавим в кортеж третье число: номер отрезка. Причём для начал отрезков будем класть номер со знаком "+", чтобы они шли в прямом порядке, а для правых концов -- со знаком "-", чтобы порядок инвертировался. Для точек третий элемент кортежа тоже пригодится: будем класть туда порядковый номер точки во входном потоке, чтобы вывести их вложенность в правильном порядке.

Для краткости я привожу решение, в котором использовал стандартную библиотечную сортировку, но у меня получалось использовать и рукописный `merge_sort` и `quick_sort`. Правда, с `quick_sort` была заковыка: обязательно надо было делить не на 2, а на 3 части. Кроме того, вариант с выбором поворотной точки по медиане не проходил 4-й тест. Выбор по рандому проходил нормально.

----

### `bisect` вне конкуренции

In [ ]:
from bisect import bisect_left, bisect_right
n, m = map(int, input().split())
lefts, rights = [], []
for _ in range(n):
    left, right = map(int, input().split())
    lefts.append(left)
    rights.append(right)
lefts.sort()
rights.sort()
print(*(bisect_right(lefts, int(dot)) - bisect_left(rights, int(dot))
        for dot in input().split()))

----

### прочие попытки...

In [1]:
def main(dots_in_cuts):
    from sys import stdin
    reader = (list(map(int, line.split())) for line in stdin)
    n_cuts, n_dots = next(reader)
    cuts = [next(reader) for _ in range(n_cuts)]
    dots = next(reader)
    assert len(dots) == n_dots
    print(*dots_in_cuts(dots, cuts))

In [10]:
def test(dots_in_cuts, nmax=10**3, niter=100, tmax=10, rseed=123):
    from random import randint, seed
    from time import perf_counter as timer
    from sys import stderr

    seed(rseed)
    
    assert dots_in_cuts([], [(1,5),(7,9)]) == []
    assert dots_in_cuts([5], []) == [0]
    assert dots_in_cuts([3], [(1,6),(5,3),(1,6)]) == [2]

    result1 = dots_in_cuts([1,6,11], [(8,9),(0,5)])
    assert result1 == [1,0,0], "got {}".format(result1)

    result2 = dots_in_cuts([1,6,11], [(0,5),(7,10)])
    assert result2 == [1,0,0], "got {}".format(result2)

    xmax = 10**6
    xmin = -xmax

    maxtime = maxiter = 0

    for t in range(niter):
        dots = [randint(xmin, xmax) for _ in range(randint(0, nmax))]
        cuts = [(min(a,b), max(a,b))
                for a,b in ((randint(xmin,xmax), randint(xmin, xmax))
                            for _ in range(randint(0, nmax)))]

        t1 = timer()
        real_result = dots_in_cuts(dots, cuts)
        t2 = timer()

        naive_result = [sum(c[0] <= d <= c[1] for c in cuts) for d in dots]

        assert real_result == naive_result
        assert t2-t1 < tmax

        maxtime = max(maxtime, t2-t1)
        maxiter = max(maxiter, (t2-t1) / (len(dots) * len(cuts) + 1))
        if t % 10 == 0:
            stderr.write(".")

    stderr.flush()
    print("ok {:.3f}ms {:.3f}us".format(maxtime*1e3, maxiter*1e6))

In [ ]:
def dots_in_cuts1(dots, cuts):
    from itertools import groupby

    if not cuts:
        return [0] * len(dots)
    if not dots:
        return []

    # 1. sort cuts by the left end
    # 2. merge identical cuts
    cl, cr, cn = zip(*((c[0], c[1], sum(1 for _ in g))
                       for c, g in groupby(sorted(cuts))
                       if c[0] <= c[1]))
    n = len(cn)-1

    counts = []
    for d in dots:
        if d < cl[0]:
            counts.append(0)
            continue

        # looking for the rightmost left end containing the dot
        # this block is O(log n_cuts)
        l, r = 0, n
        while l < r:
            # the mid point must be rounded up to avoid endless loop
            m = (l + r + 1) // 2
            if cl[m] > d:
                r = m-1
            else:
                l = m
        # the above code is equivalent to:
        if False:
            for l in range(n-1,-1,-1):
                if a[l] <= d:
                    break

        # unfortunately, this line is O(log n_cuts)
        # thus making the outer loop O(n_cuts * n_dots)
        counts.append(sum(cn[j] for j in range(l+1) if d <= cr[j]))

    return counts

test(dots_in_cuts1)

In [ ]:
ones_256 = [bin(i)[2:].count('1') for i in range(256)]
def count_ones(mask):
    cnt = 0
    while mask:
        cnt += ones_256[mask & 255]
        mask >>= 8
    return cnt

def dots_in_cuts2(dots, cuts):
    if not cuts:
        return [0] * len(dots)
    if not dots:
        return []
   
    # remove empty cuts
    mcuts = [(c[0], c[1]) for c in cuts if c[0] <= c[1]]
    n = len(mcuts)

    # cuts sorted by left end
    cl = sorted((c[0], i) for i, c in enumerate(mcuts))
    clx = [c[0] for c in cl]
    clm = []
    acc = 0
    for c in cl:
        acc += 2 ** c[1]
        clm.append(acc)

    # cuts reverse-sorted by right end
    cr = sorted(((c[1], 1<<i) for i, c in enumerate(mcuts)), reverse=True)
    crx = [c[0] for c in cr]
    crm = []
    acc = 0
    for c in cr:
        acc += c[1]
        crm.append(acc)

    counts = {d: 0 for d in dots}
    for d in counts:
        if d < cl[0][0] or d > cr[0][0]:
            counts[d] = 0
            continue

        # looking for the rightmost left end containing the dot
        l, r = 0, n-1
        while l < r:
            # the mid point must be rounded up to avoid endless loop
            m = (l + r + 1) // 2
            if clx[m] > d:
                r = m-1
            else:
                l = m
        lmask = clm[l]

        # looking for the leftmost right end containing the dot
        l, r = 0, n-1
        while l < r:
            m = (l + r + 1) // 2
            if crx[m] < d:
                r = m-1
            else:
                l = m
        rmask = crm[l]

        counts[d] = count_ones(lmask & rmask)

    return [counts[d] for d in dots]

test(dots_in_cuts2, nmax=10**4, niter=2)

In [ ]:
from bisect import bisect


def dots_in_cuts3(dots, cuts):
    if not dots:
        return []
    if not cuts:
        return [0] * len(dots)
    fcuts = [(c[0], c[1]) for c in cuts if c[0] <= c[1]]

    # cuts sorted by left end
    cl = sorted((c[0], i) for i, c in enumerate(fcuts))
    clx = [c[0] for c in cl]
    cli = [c[1] for c in cl]

    # cuts reverse-sorted by right end
    cr = sorted((-c[1], i) for i, c in enumerate(fcuts))
    crx = [c[0] for c in cr]
    cri = [c[1] for c in cr]

    hits = {d: 0 for d in dots}
    for d in hits:
        if clx[0] <= d <= -crx[0]:
            hits[d] = len(set(cli[:bisect(clx, d)]) & set(cri[:bisect(crx, -d)]))

    return [hits[d] for d in dots]


test(dots_in_cuts3, nmax=10**3, niter=100)

In [3]:
from bisect import bisect

bits_16 = tuple(bin(i).count('1') for i in range(65536))


def dots_in_cuts4(dots, cuts):
    # edge cases
    if not dots:
        return []
    if not cuts:
        return [0] * len(dots)

    # cuts sorted by left end
    cl = sorted((c[0], i) for i, c in enumerate(cuts))
    clm, mask = [], 0
    for c in cl:
        mask += 1 << c[1]
        clm.append(mask)
    clx = [c[0] for c in cl]

    # cuts reverse-sorted by right end
    cr = sorted((-c[1], i) for i, c in enumerate(cuts))
    crm, mask = [], 0
    for c in cr:
        mask += 1 << c[1]
        crm.append(mask)
    crx = [c[0] for c in cr]

    hits = {d: 0 for d in dots}
    for d in hits:
        if clx[0] <= d <= -crx[0]:
            mask = clm[bisect(clx, d) - 1] & crm[bisect(crx, -d) - 1]
            bits = 0
            while mask:
                bits += bits_16[mask & 65535]
                mask >>= 16
            hits[d] = bits

    return [hits[d] for d in dots]

test(dots_in_cuts4, nmax=10000, niter=50)

.....

ok 5082.196ms 0.215us


In [15]:
from bisect import bisect
from time import perf_counter as timer

t1 = timer()
bits_16 = tuple(bin(i).count('1') for i in range(65536))
t2 = timer()
#print('init {:.3f}s'.format(t2-t1))

def quick_sort_3(a):
    if len(a) <= 1:
        return

    cut, que = (0, len(a)), []
    while cut:
        # next cut: left, right
        l, r = cut

        # start, end, middle
        s, e = l, r-1
        m = (s + e) // 2

        # choose pivot
        i = m if a[s]<=a[m]<=a[e] else s if a[m]<=a[s]<=a[e] else e
        p = a[i]

        # init ranges: smaller, equal
        s, e = l, l+1
        a[i], a[s] = a[s], p

        # partition
        for i in range(e, r):
            v = a[i]
            if v > p:
                continue
            a[i], a[e] = a[e], p
            e += 1
            if v < p:
                a[s] = v
                s += 1

        # arrange ranges
        cut, qcut = (l, s), (e, r)
        if cut[1] - cut[0] < qcut[1] - qcut[0]:
            cut, qcut = qcut, cut

        # recurse by iteration
        if qcut[0] < qcut[1]:
            que.append(qcut)
        if cut[0] >=cut[1]:
            cut = que.pop() if que else None

#qsort = quick_sort_3
qsort = lambda a: a.sort()


def dots_in_cuts5(dots, cuts):
    # edge cases
    if not dots:
        return []
    if not cuts:
        return [0] * len(dots)

    # cuts sorted by left end
    cl = list((c[0], i) for i, c in enumerate(cuts))
    qsort(cl)
    clm, mask = [], 0
    for c in cl:
        mask += 2 ** c[1]
        clm.append(mask)
    clx = [c[0] for c in cl]
    cl.clear()

    # cuts reverse-sorted by right end
    cr = list((-c[1], i) for i, c in enumerate(cuts))
    qsort(cr)
    crm, mask = [], 0
    for c in cr:
        mask += 2 ** c[1]
        crm.append(mask)
    crx = [c[0] for c in cr]
    cr.clear()

    dd = [d for d in sorted(set(dots)) if clx[0] <= d <= -crx[0]]
    hits = {}
    for d in dd:
        mask = clm[bisect(clx, d) - 1] & crm[bisect(crx, -d) - 1]
        bits = 0
        while mask:
            bits += bits_16[mask & 65535]
            mask >>= 16
        hits[d] = bits

    return [hits.get(d, 0) for d in dots]


test(dots_in_cuts5, nmax=10000, niter=10)

init 0.042s


.

ok 1820.338ms 0.070us


In [ ]:
def dots_in_cuts6(dots, cuts):
    a = [(d, 2, i) for i, d in enumerate(dots)]
    for i, c in enumerate(cuts):
        if c[0] <= c[1]:
            a.extend(((c[0], 1, i), (c[1], 3, -i)))
    a.sort()
    hits = dots[:]
    n = 0
    for x, t, i in a:
        if t == 1:
            n += 1
        elif t == 3:
            n -= 1
        elif t == 2:
            hits[i] = n
    return hits

test(dots_in_cuts6, nmax=5*10**4, niter=10)

In [37]:
%%writefile dots_in_cuts.py
# shortest possible solution
from sys import stdin

read = lambda: map(int, stdin.readline().split())

n, m = read()
h = []

for i in range(n):
    l, r = read()
    if l <= r:
        h.append((l, -1, i))
        h.append((r, 1, -i))

for i, x in enumerate(read()):
    h.append((x, 0, i))

h.sort()

c = [0] * m
d = 0

for x, t, i in h:
    if t == 0:
        c[i] = d
    elif t < 0:
        d += 1
    else:
        d -= 1

print(*c)

Overwriting dots_in_cuts.py


In [38]:
%%bash
python3 dots_in_cuts.py << EOF
2 3
0 5
7 10
1 6 11
EOF

1 0 0


In [35]:
%%writefile dots_in_cuts.py

# with my own quicksort
from sys import stdin
from random import randint

def qsort3(a):
    if len(a) <= 1:
        return

    cut, que = (0, len(a)), []
    while cut:
        # next cut: left, right
        l, r = cut

        # start, end, middle
        s, e = l, r-1
        m = (s + e) // 2

        # choose pivot
        i = randint(s, e)
        p = a[i]

        # init ranges: smaller, equal
        s, e = l, l+1
        a[i], a[s] = a[s], p

        # partition
        for i in range(e, r):
            v = a[i]
            if v > p:
                continue
            a[i], a[e] = a[e], p
            e += 1
            if v < p:
                a[s] = v
                s += 1

        # arrange ranges
        cut, qcut = (l, s), (e, r)
        if cut[1] - cut[0] < qcut[1] - qcut[0]:
            cut, qcut = qcut, cut

        # recurse by iteration
        if qcut[0] < qcut[1]:
            que.append(qcut)
        if cut[0] >=cut[1]:
            cut = que.pop() if que else None

def read():
    return map(int, stdin.readline().split())

def main():
    n_cuts, n_dots = read()
    heap = []

    for i in range(n_cuts):
        lx, rx = read()
        if lx <= rx:
            heap.append((lx, -1, i))
            heap.append((rx, 1, -i))

    for i, dx in enumerate(read()):
        heap.append((dx, 0, i))
    assert i+1 == n_dots

    qsort3(heap)

    hits = [0] * n_dots
    depth = 0

    for x, etype, i in heap:
        if etype == 0:
            hits[i] = depth
        elif etype < 0:
            depth += 1
        else:
            depth -= 1

    print(*hits)

main()

Overwriting dots_in_cuts.py


In [36]:
%%bash
python3 dots_in_cuts.py << EOF
2 3
0 5
7 10
1 6 11
EOF

1 0 0


In [39]:
%%writefile dots_in_cuts.py

# with my own merge sort
from sys import stdin

def mergesort(a):
    s = a
    d = a[:]
    q = [(i,i) for i in range(len(a))]
    while len(q) > 1:
        p = []
        for qi in range(0,len(q)-1,2):
            li,ln = q[qi]
            ri,rn = q[qi+1]
            p.append((li,rn))
            di = li
            while li<=ln and ri<=rn:
                if s[ri] < s[li]:
                    d[di] = s[ri]
                    ri += 1
                else:
                    d[di] = s[li]
                    li += 1
                di += 1
            d[di:di+ln-li+1] = s[li:ln+1]
            d[di:di+rn-ri+1] = s[ri:rn+1]
        if len(q) % 2:
            p.append(q[-1])
            di,dn = q[-1]
            d[di:dn+1] = s[di:dn+1]
        q = p
        s,d = d,s
    if a != s:
        a[:] = s

def read():
    return map(int, stdin.readline().split())

def main():
    n_cuts, n_dots = read()
    heap = []

    for i in range(n_cuts):
        lx, rx = read()
        if lx <= rx:
            heap.append((lx, -1, i))
            heap.append((rx, 1, -i))

    for i, dx in enumerate(read()):
        heap.append((dx, 0, i))
    assert i+1 == n_dots

    mergesort(heap)

    hits = [0] * n_dots
    depth = 0

    for x, etype, i in heap:
        if etype == 0:
            hits[i] = depth
        elif etype < 0:
            depth += 1
        else:
            depth -= 1

    print(*hits)

main()

Overwriting dots_in_cuts.py


In [40]:
%%bash
python3 dots_in_cuts.py << EOF
2 3
0 5
7 10
1 6 11
EOF

1 0 0


In [ ]:
def test_bisect(nmax=500, xmax=10**3, rseed=123):
    from random import randint, seed
    import bisect
    seed(rseed)

    for n in range(1, nmax):
        if n % 100 == 0:
            print(n)  # print progress

        a = sorted(randint(1, xmax) for _ in range(n))

        for d in range(min(a)-3, max(a)+3):
            # binary lookup
            l, r = 0, n-1
            while l < r:
                m = (l + r + 1) // 2
                if a[m] > d:
                    r = m-1
                else:
                    l = m

            # linear lookup
            for i in range(n-1,-1,-1):
                if a[i] <= d:
                    break

            # bisect
            b = bisect.bisect(a, d) - 1
                    
            assert i == l == b, "a={} d={} n={} l={} i={} b={}".format(a, d, n, l, i, b)

    print("ok")

test_bisect()

In [ ]:
set([1,5,2,3]) & set([4,5,6,2])